In [6]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [12]:
import csv, datetime, fcntl, glob, json, math, os, re, subprocess, sys, time, urllib2, xml.dom.minidom
from dateutil import rrule
from dateutil import tz
import dateutil.parser
import numpy as np

# To install dateutil on Ubuntu
# sudo pip install python-dateutil

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('python-utils/esdr-library.ipynb')

In [2]:
esdr = Esdr(None)
product = esdr.get_product_by_name('PurpleAir')
product

{u'created': u'2017-11-20T15:17:13.000Z',
 u'creatorUserId': 3,
 u'defaultChannelSpecs': {},
 u'description': u'PurpleAir',
 u'id': 69,
 u'modified': u'2017-11-20T15:17:13.000Z',
 u'name': u'PurpleAir',
 u'prettyName': u'PurpleAir',
 u'vendor': u'PurpleAir'}

In [3]:
feeds = []
# Approx bounding box for Allegheny County
region = 'latitude>40.192204,latitude<40.674084,longitude<-79.688618,longitude>-80.361022'
offset = 0
while True:
    response = esdr.api('GET', '/api/v1/feeds', {'whereAnd' : 'productId=%d,%s' % (product['id'], region), 'offset': str(offset)})
    data = response['data']
    totalCount = data['totalCount']
    nrecs = len(data['rows'])
    feeds.extend(data['rows'])
    limit = data['limit']
    total_recvd = offset + nrecs
    #print(offset, nrecs, limit, totalCount)
    if total_recvd >= totalCount:
        break
    offset += nrecs

print 'Retrieved %d feeds' % len(feeds)
print 'First feed: %s' % feeds[0]

Retrieved 153 feeds
First feed: {u'apiKeyReadOnly': u'2bd4d40a3059b5ef56a3f6a6b78386569da2c2580fcd5c42e497072119f7b79c', u'minTimeSecs': 1522247982.349, u'name': u'Monaca,PA PurpleAir', u'created': u'2018-03-28T14:51:40.000Z', u'isPublic': 1, u'userId': 3, u'modified': u'2018-08-30T13:39:59.000Z', u'longitude': -80.311057, u'channelBounds': {u'channels': {u'PM2_5': {u'maxTimeSecs': 1535635729.924, u'minTimeSecs': 1522247982.349, u'maxValue': 3810.59, u'minValue': 0}, u'stats_v': {u'maxTimeSecs': 1535635729.924, u'minTimeSecs': 1522247982.349, u'maxValue': 970.64, u'minValue': 0}, u'stats_pm': {u'maxTimeSecs': 1535635729.924, u'minTimeSecs': 1522247982.349, u'maxValue': 970.64, u'minValue': 0}, u'temp_f': {u'maxTimeSecs': 1535635729.924, u'minTimeSecs': 1522247982.349, u'maxValue': 104, u'minValue': 31}, u'humidity': {u'maxTimeSecs': 1535635729.924, u'minTimeSecs': 1522247982.349, u'maxValue': 84, u'minValue': 11}, u'pressure': {u'maxTimeSecs': 1535635729.924, u'minTimeSecs': 1522247982

In [14]:
def compute_row(feed, nbins, from_epoch, to_epoch):
    feed_id = feed['id']
    export_url = 'https://esdr.cmucreatelab.org/api/v1/feeds/%d/channels/PM2_5/export?from=%d&to=%d&format=json' % (feed_id, from_epoch, to_epoch)
    data = json.load(urllib2.urlopen(export_url))['data']
    print('Feed %d has %d samples' % (feed_id, len(data)))
    if len(data) == 0:
        print('   No samples, skipping')
        return None
    sums = np.zeros(nbins)
    counts = np.zeros(nbins)
    for sample in data:
        idx = bin(sample[0]) - first_bin
        sums[idx] += sample[1]
        counts[idx] += 1
    dat = np.divide(sums, np.maximum(counts, 1))
    if numpy.median(dat) == 0:
        print('   Median is zero, skipping')
        return None
    print('   %f,%f,%f' % (numpy.min(dat), numpy.median(dat), numpy.max(dat)))
    if numpy.min(dat) < -5000:
        print('   Min is too low, skipping')
        return None
    dat -= numpy.median(dat)
    row = ['feed_%d' % feed_id, feed['latitude'], feed['longitude']] + list(dat)
    return ','.join([str(z) for z in row])

nbins = 24 * 12 # every 5 mins
compute_row(feeds[110], nbins, 1535515200, 1535515200+86400)

Feed 17984 has 63 samples


TypeError: 'float' object cannot be interpreted as an index

In [27]:
z['data'].keys()

[u'totalCount', u'rows', u'limit', u'offset']

In [19]:
np.array([1.0,2,3,4,5]).astype(np.float32)

array([ 1.,  2.,  3.,  4.,  5.], dtype=float32)

In [20]:
np.array([1.0,2,3,4,5]).astype(np.float32).tobytes()

'\x00\x00\x80?\x00\x00\x00@\x00\x00@@\x00\x00\x80@\x00\x00\xa0@'

In [24]:
json.dumps(list(np.array([1.0,2,3,4,5]).astype(np.float64)))

'[1.0, 2.0, 3.0, 4.0, 5.0]'